# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


C:\Users\kjohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [3]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df.prestige, df.admit)

admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [9]:
#df = pd.get_dummies(df, columns=['prestige'], drop_first=True)
dummy_ranks = pd.get_dummies(df['prestige'], prefix='prestige', drop_first=True)

In [10]:
dummy_ranks.head()

,prestige_2.0,prestige_3.0,prestige_4.0
0,0,1,0
1,0,1,0
2,0,0,0
3,0,0,1
4,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: 6 (this question's wording is a little confusing to me) 

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [11]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


C:\Users\kjohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


In [8]:
handCalc.head()

,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


In [12]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(df.prestige, df.admit)

admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [61]:
#df.groupby('prestige')['admit'].sum()[1]

In [28]:
print 33/51.0

0.647058823529


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [37]:
a=53+28+12
b = a + 95+93+55 
1.0*a/(b)

0.2767857142857143

#### 3.3 Calculate the odds ratio

In [42]:
(33.0/51)/(1.0*a/b)


2.3377609108159394

#### 3.4 Write this finding in a sentenance: 

Answer: Going from prestige 2 to 1 results in a 2.33x increase in the odds of getting admitted

#### 3.5 Print the cross tab for prestige_4

In [65]:
pd.crosstab(handCalc['prestige_4.0'], handCalc.admit)

admit,0,1
prestige_4.0,,
0,216,114
1,55,12


#### 3.6 Calculate the OR 

In [49]:
(12.0/67)/(114.0/330)

0.5184603299293008

#### 3.7 Write this finding in a sentence

Answer: The odds of gaining admittance are 50% as likely for a prestige 4 school vs any other prestige.

## Part 4. Analysis

In [66]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [67]:
# manually add the intercept
data['intercept'] = 1.0

In [68]:
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0  intercept
0      0  380.0  3.61             0             1             0        1.0
1      1  660.0  3.67             0             1             0        1.0
2      1  800.0  4.00             0             0             0        1.0
3      1  640.0  3.19             0             0             1        1.0
4      0  520.0  2.93             0             0             1        1.0


#### 4.1 Set the covariates to a variable called train_cols

In [70]:
#features = [i for i in df.columns if 'dow_' in i]
train_cols = data.columns[1:]
print train_cols

Index([u'gre', u'gpa', u'prestige_2.0', u'prestige_3.0', u'prestige_4.0',
       u'intercept'],
      dtype='object')


#### 4.2 Fit the model

In [88]:
# import numpy as np
# import statsmodels.api as sm
mod = sm.OLS(data['admit'], data[train_cols]).fit()
# res = mod.fit()


#### 4.3 Print the summary results

In [112]:
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  admit   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     8.594
Date:                Wed, 30 Aug 2017   Prob (F-statistic):           9.71e-08
Time:                        01:03:23   Log-Likelihood:                -239.02
No. Observations:                 397   AIC:                             490.0
Df Residuals:                     391   BIC:                             513.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0004      0.000      1.997      0.047    6.48e-06       0.001
gpa              0.1508      0.064      2.349      0.019       0.025       0.277
prestige_2.0    -0.1635      0.068     -2.407      0.017      -0.297      -0.030
prestige_3.0    -0.2910      0.070     -4.139      0.000      -0.429      -0.153
prestige_4.0    -0.3240      0.079     -4.082      0.000      -0.480      -0.168
intercept       -0.2377      0.217     -1.095      0.274      -0.665       0.189
==============================================================================
Omnibus:                      152.312   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.314
Skew:                           0.678   Prob(JB):                     1.19e-11
Kurtosis:                       1.904   Cond. No.                     6.07e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.07e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [132]:
# couldn't figure out how to pull the params and conf_int simply (mod.conf_int() was not cooperating)
d = ['params': mod.params, 'two_point_5':mod.conf_int()[0], 'nine_sev_five':mod.conf_int()[1]]
# conf[]=mod.params
# a=params
# b=mod.conf_int()[0]
# c=mod.conf_int()[1]
# a.append(mod.conf_int()[1])
# print a, b, c
# mod.params
# mod.params
# print low, high

SyntaxError: invalid syntax (<ipython-input-132-0016331146c9>, line 2)

In [159]:
odds_ratios = np.exp(mod.params)
# odds_ratios.append(np.exp(mod.conf_int()))
a = mod.conf_int()
a['params'] = mod.params
print np.exp(a)
# print np.exp(odds_ratios)

                     0         1    params
gre           1.000006  1.000837  1.000422
gpa           1.024866  1.319280  1.162792
prestige_2.0  0.742965  0.970467  0.849131
prestige_3.0  0.651050  0.858342  0.747545
prestige_4.0  0.618748  0.845411  0.723254
intercept     0.514525  1.208055  0.788400


#### 4.5 Interpret the OR of Prestige_2

Answer: Going to a prestige 2 school changes odds of admittance by .84

#### 4.6 Interpret the OR of GPA

Answer: A 1 unit increase in GPA increases odds of admittance by 1.16 times.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [166]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [177]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))
combos.columns=['gre','gpa','prestige','admit']

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [193]:
# recreate the dummy variables
cols_to_keep = ['admit', 'gre', 'gpa']
data = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])

# keep only what we need for making predictions


#### 5.2 Make predictions on the enumerated dataset

In [202]:
mod.predict(data[['gre','gpa','prestige_2.0','prestige_3.0','prestige_4.0','admit']])
# mod.summary()

0     -0.095102
1     -0.095102
2      0.195859
3     -0.128135
4     -0.098976
5      0.061476
6      0.225018
7      0.061476
8     -0.036783
9      0.090636
10    -0.069817
11     0.254178
12     0.283337
13     0.119795
14     0.283337
15    -0.007624
16    -0.011498
17     0.021535
18     0.148954
19     0.312497
20     0.050695
21     0.178114
22     0.017662
23     0.017662
24     0.207273
25     0.370815
26     0.370815
27     0.046821
28     0.236432
29     0.399975
         ...   
370    0.335148
371    0.207729
372    0.527850
373    0.527850
374    0.364308
375    0.203855
376    0.393467
377    0.393467
378    0.266048
379    0.393467
380    0.422626
381    0.422626
382    0.422626
383    0.586168
384    0.451786
385    0.615328
386    0.451786
387    0.451786
388    0.480945
389    0.480945
390    0.480945
391    0.480945
392    0.382685
393    0.510104
394    0.382685
395    0.510104
396    0.411845
397    0.539264
398    0.539264
399    0.411845
Length: 400, dtype: floa

#### 5.3 Interpret findings for the last 4 observations

Answer: row 396 and 399 are <.5, so won't get in. 397 and 398 will.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.